## Setup
### Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import sys
import os
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

IS_COLAB = 'google.colab' in sys.modules
if IS_COLAB:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
from pprint import pprint
import time
import matplotlib.pyplot as plt

%matplotlib notebook

#### Scikit-optimizer

In [2]:
import skopt
# !pip install scikit-optimize if  necessary
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer  

/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


### Setup GPU

In [3]:
# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


### Constants

In [4]:
SEED = 1234
np.random.seed(SEED)
tf.random.set_seed(SEED)

if IS_COLAB:
    DATASET_PATH = '/content/drive/My Drive'
else:
    DATASET_PATH = '/kaggle/input/ann-and-dl-image-segmentation'

CWD = os.getcwd()
DATASET_DIR_NAME = 'Segmentation_Dataset'
EXPERIMENTS_PATH = os.path.join(CWD, 'segmentation_experiments')

EPOCHS = 150
LEARNING_RATE = 1e-3
SHOW_PLOTS = False
USE_DATA_AUGMENTATION = False
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 4
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
OUTPUT_CHANNELS = 1
RGB_CHANNELS = 3
NUMBER_OF_CLASSES = 1 # house / not-house
NN_DEPTH = 8
STARTING_FILTERS = 4
EARLY_STOP = True
SAVE_PREDICTIONS = True
SHOW_PREDICTED_MASKS = False
FIND_HYPERPARAMETERS = True
DYNAMIC_SHAPE=False
HP_CALLS = 11

print("cwd: %s" % CWD)

cwd: /kaggle/working


##### Optimizer

In [5]:
dim_learning_rate = Real(low=1e-4, high=1e-2, prior='log-uniform', name='learning_rate')
dim_epochs = Integer(low=100, high=200, name="epochs")
dim_batch_size = Integer(low=2, high=64, name="batch_size")
dim_nn_depth = Integer(low=1, high=8, name="nn_depth")
dim_starting_filters = Integer(low=1, high=4, name="starting_filters")
dim_activation_fn = Categorical(categories=['relu', 'sigmoid'], name='activation_fn')
dim_loss = Categorical(categories=['binary_crossentropy', 'sparse_categorical_crossentropy', 'dice_loss'], name='loss_fn')
dim_optimizer = Categorical(categories=['adam', 'sgd'], name='optimizer_fn')

dimensions = [dim_learning_rate, dim_epochs, dim_batch_size, dim_nn_depth, dim_starting_filters, dim_activation_fn, dim_loss, dim_optimizer]
default_parameters= [LEARNING_RATE, EPOCHS, BATCH_SIZE, NN_DEPTH, STARTING_FILTERS, 'relu', 'binary_crossentropy', 'adam']


## Data Generators

In [6]:
if USE_DATA_AUGMENTATION:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='constant',
                                            cval=0,
                                            rescale=1. / 255,
                                            validation_split=VALIDATION_SPLIT)

    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='constant',
                                             cval=0,
                                             rescale=1. / 255,
                                             validation_split=VALIDATION_SPLIT)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1. / 255, validation_split=VALIDATION_SPLIT)
    train_mask_data_gen = ImageDataGenerator(rescale=1. / 255, validation_split=VALIDATION_SPLIT)

test_img_data_gen = ImageDataGenerator(rescale=1./255)

# Create generators to read images from dataset directory
# -------------------------------------------------------

# path to dataset
dataset_dir = os.path.join(DATASET_PATH, DATASET_DIR_NAME)
# path to train dataset
train_dir = os.path.join(dataset_dir, 'training')
# path to test dataset
test_dir = os.path.join(dataset_dir, 'test')

print('dataset %s \ntrain dir %s \ntest dir %s \n' % (dataset_dir, train_dir, test_dir))

dataset /kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset 
train dir /kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset/training 
test dir /kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset/test 



### Dataset generation

In [7]:
## Training images
train_img_gen = train_img_data_gen.flow_from_directory(
    os.path.join(train_dir, 'images'),
    subset='training',  # subset of data
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
#     color_mode='grayscale',
    class_mode=None,
    shuffle=True,
    interpolation='bilinear',
    seed=SEED)
## Training masks
train_mask_gen = train_mask_data_gen.flow_from_directory(
    os.path.join(train_dir, 'masks'),
    subset='training',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode=None,
    shuffle=True,
    interpolation='bilinear',
    seed=SEED)

## Validation images
valid_img_gen = train_img_data_gen.flow_from_directory(
    os.path.join(train_dir, 'images'),
    subset='validation',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
#     color_mode='grayscale',
    class_mode=None,
    shuffle=False,
    interpolation='bilinear',
    seed=SEED)

## Validation masks
valid_mask_gen = train_mask_data_gen.flow_from_directory(
    os.path.join(train_dir, 'masks'),
    subset='validation',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode=None,
    shuffle=False,
    interpolation='bilinear',
    seed=SEED)

## Test images
test_img_gen = test_img_data_gen.flow_from_directory(os.path.join(test_dir, 'images'),
                                                     target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                                     batch_size=BATCH_SIZE, 
                                                     class_mode=None, # Because we have no class subfolders in this case
                                                     shuffle=False,
                                                     interpolation='bilinear',
                                                     seed=SEED)

## Validation generator
valid_gen = zip(valid_img_gen, valid_mask_gen)
## Training generator
train_gen = zip(train_img_gen, train_mask_gen)
## Test generator ==> since we dont have masks..
test_gen = test_img_gen

Found 6118 images belonging to 1 classes.
Found 6118 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.
Found 1529 images belonging to 1 classes.
Found 1234 images belonging to 1 classes.


### Create dataset objects

In [8]:
def prepare_target(x_, y_):
    y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), tf.float32)
    return x_, tf.where(y_ > 0, y_ - 1, y_ + 1)

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS], [None, IMAGE_HEIGHT, IMAGE_WIDTH, OUTPUT_CHANNELS]))
train_dataset = train_dataset.map(prepare_target)
# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS], [None, IMAGE_HEIGHT, IMAGE_WIDTH, OUTPUT_CHANNELS]))
valid_dataset = valid_dataset.map(prepare_target)
# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----------
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS], [None, IMAGE_HEIGHT, IMAGE_WIDTH, OUTPUT_CHANNELS]))
test_dataset = test_dataset.map(prepare_target)
# Repeat
test_dataset = test_dataset.repeat()

#### Test data generator

In [9]:
if SHOW_PLOTS:
    fig, ax = plt.subplots(1, 2)
    fig.show()

    # Assign a color to each class
    colors_dict = {}
    colors_dict[0] = [255, 255, 255]  # foreground
    colors_dict[1] = [0, 0, 0]  # background
    colors_dict[2] = [3, 82, 252] # contours

    iterator = iter(train_dataset)

    for _ in range(2):
        augmented_img, target = next(iterator)
        augmented_img = augmented_img[0]   # First element
        augmented_img = augmented_img * 255  # denormalize

        target = np.array(target[0, ..., 0])   # First element (squeezing channel dimension)

        # Assign colors (just for visualization)
        target_img = np.zeros([target.shape[0], target.shape[1], 3])

        target_img[np.where(target == 0)] = colors_dict[0]
        target_img[np.where(target == 1)] = colors_dict[1]
        target_img[np.where(target == 2)] = colors_dict[2]

        ax[0].imshow(np.uint8(augmented_img))
        ax[1].imshow(np.uint8(target_img))

        fig.canvas.draw()
        time.sleep(1)

## Convolutional Neural Network
### Functions

In [10]:
def dice_coeficcient(y_true, y_pred, smooth=1):
    from keras import backend as K
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

# Dice loss
def dice_loss(y_true, y_pred):
    return 1-dice_coeficcient(y_true, y_pred)

# IoU metric function
def iou_metric(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax
    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

# Create a model 
def create_model(depth, start_f, num_classes, dynamic_input_shape, activation_fn='relu'):
    model = tf.keras.Sequential()

    # Encoder
    # -------
    for i in range(depth):
        if i == 0:
            if dynamic_input_shape:
                input_shape = [None, None, RGB_CHANNELS]
            else:
                input_shape = [IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS]
        else:
            input_shape = [None]

        model.add(
            tf.keras.layers.Conv2D(filters=start_f,
                                   kernel_size=(3, 3),
                                   strides=(1, 1),
                                   padding='same',
                                   input_shape=input_shape, 
                                   activation=activation_fn))
#         model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

        start_f *= 2

    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(
            tf.keras.layers.Conv2D(filters=start_f // 2,
                                   kernel_size=(3, 3),
                                   strides=(1, 1),
                                   padding='same',
                                   activation=activation_fn))

#         model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(
        tf.keras.layers.Conv2D(filters=num_classes,
                               kernel_size=(1, 1),
                               strides=(1, 1),
                               padding='same',
                               activation='sigmoid'))

    return model

### Find Hyperparameters
> dimensions = [dim_learning_rate, dim_epochs, dim_batch_size, dim_nn_depth, dim_starting_filters, dim_activation_fn, dim_loss]
> default_parameters= [LEARNING_RATE, EPOCHS, BATCH_SIZE, NN_DEPTH, STARTING_FILTERS, 'relu', 'binary_crossentropy']

In [11]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, epochs, batch_size, nn_depth, starting_filters, activation_fn, loss_fn, optimizer_fn):
    from tensorflow.python.keras import backend as K
    
    number_of_classes = NUMBER_OF_CLASSES
    if loss_fn is 'sparse_categorical_crossentropy':
        number_of_classes = 2
    
    model = create_model(depth=nn_depth, 
                         start_f=starting_filters, 
                         num_classes=number_of_classes, 
                         dynamic_input_shape=DYNAMIC_SHAPE, 
                         activation_fn=activation_fn)
    
    # optimizer
    if optimizer_fn is 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_fn is 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.5, nesterov=True)
        
    # Validation metrics
    metrics = [iou_metric, 'accuracy']

    loss_function = loss_fn
    if loss_fn is 'dice_loss':
        # function pointer to dice_loss
        loss_function = dice_loss
    
    # Compile Model
    model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)
    
    callbacks= []
    if EARLY_STOP:
        es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_iou_metric', patience=10)
        callbacks.append(es_callback)

        
    print('current parameters: \n learning_rate: %d,\n epochs: %d,\n batch_size: %d,\n nn_depth: %d,\n starting_filters: %d,\n activation_fn: %s,\n loss_fn: %s,\n optimizer_fn: %s' 
          % (learning_rate, epochs, batch_size, nn_depth, starting_filters, activation_fn, loss_fn, optimizer_fn))
    blackbox = model.fit(x=train_dataset, epochs=epochs, 
                          steps_per_epoch = (len(train_img_gen)//batch_size),
                          validation_data=valid_gen, 
                          validation_steps=(len(valid_img_gen)//batch_size), callbacks=callbacks)
    
    #return the validation accuracy for the last epoch.
    accuracy = blackbox.history['val_iou_metric'][-1]

    # Print the classification accuracy.
    print()
    print("IoU metric: {0:.2%}".format(accuracy))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
#     tensorflow.reset_default_graph()
    
    # the optimizer aims for the lowest score, so we return our negative accuracy
    return -accuracy

In [ ]:
if FIND_HYPERPARAMETERS:
    gp_result = gbrt_minimize(func=fitness,
                              dimensions=dimensions,
                              n_calls=HP_CALLS,
                              n_jobs=-1,
                              x0=default_parameters)

current parameters: 
 learning_rate: 0,
 epochs: 150,
 batch_size: 4,
 nn_depth: 8,
 starting_filters: 4,
 activation_fn: relu,
 loss_fn: binary_crossentropy,
 optimizer_fn: adam
Train for 382 steps, validate for 95 steps
Epoch 1/150
382/382 [==============================] - 20s 53ms/step - loss: 0.6361 - iou_metric: 0.6869 - accuracy: 0.6877 - val_loss: 0.8189 - val_iou_metric: 0.3772 - val_accuracy: 0.3772
Epoch 2/150
382/382 [==============================] - 15s 39ms/step - loss: 0.6158 - iou_metric: 0.6973 - accuracy: 0.6973 - val_loss: 0.9181 - val_iou_metric: 0.2801 - val_accuracy: 0.2801
Epoch 3/150
382/382 [==============================] - 15s 39ms/step - loss: 0.6151 - iou_metric: 0.6961 - accuracy: 0.6961 - val_loss: 0.9209 - val_iou_metric: 0.2538 - val_accuracy: 0.2538
Epoch 4/150
382/382 [==============================] - 14s 37ms/step - loss: 0.6157 - iou_metric: 0.6963 - accuracy: 0.6963 - val_loss: 1.0236 - val_iou_metric: 0.3063 - val_accuracy: 0.3077
Epoch 5/150
38

Using TensorFlow backend.


current parameters: 
 learning_rate: 0,
 epochs: 127,
 batch_size: 55,
 nn_depth: 5,
 starting_filters: 3,
 activation_fn: relu,
 loss_fn: dice_loss,
 optimizer_fn: adam
Train for 27 steps, validate for 6 steps
Epoch 1/127
27/27 [==============================] - 2s 88ms/step - loss: 0.1358 - iou_metric: 0.6864 - accuracy: 0.6864 - val_loss: 0.2149 - val_iou_metric: 0.2735 - val_accuracy: 0.2735
Epoch 2/127
27/27 [==============================] - 1s 28ms/step - loss: 0.1293 - iou_metric: 0.6835 - accuracy: 0.6835 - val_loss: 0.2504 - val_iou_metric: 0.2399 - val_accuracy: 0.2399
Epoch 3/127
27/27 [==============================] - 1s 27ms/step - loss: 0.1243 - iou_metric: 0.7000 - accuracy: 0.7000 - val_loss: 0.2310 - val_iou_metric: 0.2927 - val_accuracy: 0.2927
Epoch 4/127
27/27 [==============================] - 1s 27ms/step - loss: 0.1244 - iou_metric: 0.6991 - accuracy: 0.6991 - val_loss: 0.2629 - val_iou_metric: 0.2151 - val_accuracy: 0.2151
Epoch 5/127
27/27 [==================

24/24 [==============================] - 1s 27ms/step - loss: 0.5515 - iou_metric: 0.7022 - accuracy: 0.7022 - val_loss: 1.2358 - val_iou_metric: 0.2579 - val_accuracy: 0.2579
Epoch 20/128
24/24 [==============================] - 1s 27ms/step - loss: 0.5942 - iou_metric: 0.6755 - accuracy: 0.6755 - val_loss: 1.2197 - val_iou_metric: 0.3154 - val_accuracy: 0.3154
Epoch 21/128
24/24 [==============================] - 1s 27ms/step - loss: 0.5813 - iou_metric: 0.6859 - accuracy: 0.6859 - val_loss: 1.3387 - val_iou_metric: 0.2586 - val_accuracy: 0.2586

IoU metric: 25.86%

current parameters: 
 learning_rate: 0,
 epochs: 180,
 batch_size: 7,
 nn_depth: 3,
 starting_filters: 4,
 activation_fn: relu,
 loss_fn: sparse_categorical_crossentropy,
 optimizer_fn: sgd
Train for 218 steps, validate for 54 steps
Epoch 1/180
218/218 [==============================] - 7s 31ms/step - loss: 0.6316 - iou_metric: 0.7168 - accuracy: 0.6924 - val_loss: 0.9255 - val_iou_metric: 0.2828 - val_accuracy: 0.2828
Ep

In [ ]:
print('Best parameters found:\n\tlearning_rate: %f,\n\tepochs: %d,\n\tbatch_size: %d,\n\tnn_depth: %d,\n\tstarting_filters: %d,\n\tactivation_fn: %s,\n\tloss_fn: %s,\n\toptimizer_fn: %s' 
          % (gp_result.x[0], gp_result.x[1], gp_result.x[2], gp_result.x[3], gp_result.x[4], gp_result.x[5], gp_result.x[6], gp_result.x[7]))

### Create the model

In [ ]:
number_of_classes = NUMBER_OF_CLASSES
if gp_result.x[6] is 'sparse_categorical_crossentropy':
    number_of_classes = 2
        
if FIND_HYPERPARAMETERS:
#     [dim_learning_rate, dim_epochs, dim_batch_size, dim_nn_depth, dim_starting_filters, dim_activation_fn, dim_loss]
    model = create_model(depth=gp_result.x[3], 
                         start_f=gp_result.x[4], 
                         num_classes=number_of_classes, 
                         dynamic_input_shape=DYNAMIC_SHAPE, 
                         activation_fn=gp_result.x[5])
else:
    model = create_model(depth=NN_DEPTH,
                         start_f=STARTING_FILTERS,
                         num_classes=number_of_classes,
                         dynamic_input_shape=DYNAMIC_SHAPE)

# Visualize created model as a table
model.summary()

# Visualize initialized weights
# print(model.weights)

### Compile the model

In [ ]:
if FIND_HYPERPARAMETERS:
#     [dim_learning_rate, dim_epochs, dim_batch_size, dim_nn_depth, dim_starting_filters, dim_activation_fn, dim_loss]
    loss = gp_result.x[6]
    if gp_result.x[7] is 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=gp_result.x[0])
    elif gp_result.x[7] is 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=gp_result.x[0], momentum=0.5, nesterov=True)
else:
    # Loss
    # Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
    # loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    loss = 'binary_crossentropy'
    # optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    
# Validation metrics
metrics = [iou_metric, 'accuracy']
# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training

In [ ]:
callbacks = []

if not os.path.exists(EXPERIMENTS_PATH):
    os.makedirs(EXPERIMENTS_PATH)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(EXPERIMENTS_PATH, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

# Model checkpoint
# ----------------
# ckpt_dir = os.path.join(exp_dir, 'ckpts')
# if not os.path.exists(ckpt_dir):
#     os.makedirs(ckpt_dir)

# ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
#                                                    save_weights_only=False)  # False to save the model directly
# callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
# tb_dir = os.path.join(exp_dir, 'tb_logs')
# if not os.path.exists(tb_dir):
#     os.makedirs(tb_dir)
    
# # By default shows losses and metrics for both training and validation
# tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
#                                              profile_batch=0,
#                                              histogram_freq=0)  # if 1 shows weights histograms
# callbacks.append(tb_callback)

# Early Stopping
# --------------
if EARLY_STOP:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_iou_metric', patience=10)
    callbacks.append(es_callback)

if FIND_HYPERPARAMETERS:
    EPOCHS = gp_result.x[1]
    BATCH_SIZE = gp_result.x[2]
    
model.fit(x=train_dataset, epochs=EPOCHS, 
                          steps_per_epoch = (len(train_img_gen)//BATCH_SIZE),
                          validation_data=valid_gen, 
                          validation_steps=(len(valid_img_gen)//BATCH_SIZE), 
                          callbacks=callbacks)


## Evaluation

In [ ]:
evaluation_output = model.evaluate(x=valid_dataset, steps=len(valid_img_gen), verbose=1)

print('Loss: %s \nIoU Metric: %s \nAccuracy: %s' % (evaluation_output[0], evaluation_output[1], evaluation_output[2]))

## Final result
### Functions

In [ ]:
def calculate_competition_score(score):
    thresholds = np.arange(0.5, 1.0, 0.05)
    competition_score = 0

    for t in thresholds:
        if score > t:
            competition_score += 1

    competition_score /= len(thresholds)

    return competition_score

# Compute predictions
def generate_predictions(model):
    # Cycle over test images
    test_img_dir = os.path.join(test_dir, 'images', 'img')

    # s[:10] predict until 10th image
    image_filenames = next(os.walk(test_img_dir))[2]

    results = {}
    masks = {}

    for filename in image_filenames:
        # test images are in RGB, hence no need to transform them.
        img = Image.open(os.path.join(test_img_dir, filename))
        img = img.resize((IMAGE_HEIGHT, IMAGE_WIDTH))  # target size

        # data_normalization
        img_array = np.array(img)  #
        # img_array = img_array * 1. / 255  # normalization
        img_array = np.expand_dims(img_array, axis=0) # to fix dims of input in the model

        # print("prediction for {}...".format(filename))
        predictions = model.predict(img_array)

        # Get predicted class as the index corresponding to the maximum value in the vector probability
#         predicted_mask = tf.argmax(predictions, axis=-1)
#         predicted_mask = predicted_mask[0]
#         target = np.array(predicted_mask)
#         target = rle_encode(target)
        predictions = np.round(predictions)
        predicted_mask = predictions[0]
        target = rle_encode(predicted_mask)
        
        # print(target.shape)
        results[filename[:-4]] = target
        masks[filename[:-4]] = predicted_mask

    # create_csv(results)
    print('Num. of labeled images', results.__len__())
    return results, masks

def show_results(results, limit=2):
    fig, ax = plt.subplots(1, 2)
    fig.show()

    # Assign a color to each class
    colors_dict = {}
    colors_dict[0] = [252, 186, 3]  # foreground
    colors_dict[1] = [0, 0, 0]  # background
    colors_dict[2] = [3, 82, 252] # contours

    for file, mask in results.items():
        filename = file + '.tif'
        img_path = os.path.join(test_dir, 'images/img')
        test_img = Image.open(os.path.join(img_path,filename)).convert('RGB')  # open as RGB
        test_img = test_img.resize((IMAGE_HEIGHT, IMAGE_WIDTH))  # target size
        
        target = np.array(mask)   # First element (squeezing channel dimension)
#         print(target.shape)
        # Assign colors (just for visualization)
        target_img = np.zeros([target.shape[0], target.shape[1], 3])

#         target_img[np.where(target == 0)] = colors_dict[0]
#         target_img[np.where(target == 1)] = colors_dict[1]
#         target_img[np.where(target == 2)] = colors_dict[2]
        
#         print(target_img)

        ax[0].imshow(np.uint8(test_img))
        ax[1].imshow(np.uint8(target_img))

        fig.canvas.draw()
        time.sleep(1)
    
## Create CSV file
def create_csv(results, results_dir='./'):
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'
    csv_fname = os.path.join(results_dir, csv_fname)
    with open(csv_fname, 'w') as f:
        f.write('ImageId,EncodedPixels,Width,Height\n')
        for key, value in results.items():
            f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')
        
def rle_encode(img):
    # Flatten column-wise
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
        

### Generate predictions

In [ ]:
predictions, masks = generate_predictions(model)

In [ ]:
if SHOW_PREDICTED_MASKS:
    show_results(masks)

#### Save predictions

In [ ]:
if SAVE_PREDICTIONS:
    print('saving predictions....')
    predictions_dir = os.path.join(CWD, 'predictions')
    if not os.path.exists(predictions_dir):
        os.makedirs(predictions_dir)
    create_csv(predictions, predictions_dir)
else:
#     competition score based on evaluation.
    calculate_competition_score(evaluation_output[1])

In [ ]:
# test_img_dir = os.path.join(test_dir, 'images', 'img')
# # test images are in RGB, hence no need to transform them.
# img = Image.open(os.path.join(test_img_dir, '1155.tif'))
# img = img.resize((IMAGE_HEIGHT, IMAGE_WIDTH))  # target size

# # data_normalization
# img_array = np.array(img)  #
# # img_array = img_array * 1. / 255  # normalization
# img_array = np.expand_dims(img_array, axis=0) # to fix dims of input in the model

# # print("prediction for {}...".format(filename))
# prediction_1155 = model.predict(img_array / 255.)

In [ ]:
# prediction_1155 = np.round(prediction_1155)
# prediction_1155[0]